## 175. Combine Two Tables (Easy)

Table: `Person`
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| PersonId    | int     |
| FirstName   | varchar |
| LastName    | varchar |
+-------------+---------+
```
PersonId is the primary key column for this table.
Table: `Address`
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| AddressId   | int     |
| PersonId    | int     |
| City        | varchar |
| State       | varchar |
+-------------+---------+
```
`AddressId` is the primary key column for this table.


Write a SQL query for a report that provides the following information for each person in the `Person` table, regardless if there is an address for each of those people:

`FirstName`, `LastName`, `City`, `State`

```mysql
SELECT p.FirstName, p.LastName, a.City, a.State
FROM Person p
LEFT JOIN Address a ON a.PersonId = p.PersonId;
```

## 176. Second Highest Salary (Easy)
Write a SQL query to get the second highest salary from the `Employee` table.
```
+----+--------+
| Id | Salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
```
For example, given the above `Employee` table, the query should return 200 as the second highest salary. If there is no second highest salary, then the query should return null.
```
+---------------------+
| SecondHighestSalary |
+---------------------+
| 200                 |
+---------------------+
```

```mysql
select
    (select distinct salary
    from employee
    order by salary desc
    limit 1, 1) as SecondHighestSalary;
```


## 177. Nth Highest Salary (Medium)
Write a SQL query to get the nth highest salary from the `Employee` table.
```
+----+--------+
| Id | Salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
```
For example, given the above `Employee` table, the nth highest salary where n = 2 is 200. If there is no nth highest salary, then the query should return null.
```
+------------------------+
| getNthHighestSalary(2) |
+------------------------+
| 200                    |
+------------------------+
```




```mysql
create function getNthHighestSalary(N int) returns int
begin
declare m int;
set m=n-1;
return (
    (select distinct salary
    from Employee
    order by salary desc
    limit m, 1)
);
end
```

## 185. Department Top Three Salaries (Hard)
The `Employee` table holds all employees. Every employee has an Id, and there is also a column for the department Id.
```
+----+-------+--------+--------------+
| Id | Name  | Salary | DepartmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 85000  | 1            |
| 2  | Henry | 80000  | 2            |
| 3  | Sam   | 60000  | 2            |
| 4  | Max   | 90000  | 1            |
| 5  | Janet | 69000  | 1            |
| 6  | Randy | 85000  | 1            |
| 7  | Will  | 70000  | 1            |
+----+-------+--------+--------------+
```
The `Department` table holds all departments of the company.
```
+----+----------+
| Id | Name     |
+----+----------+
| 1  | IT       |
| 2  | Sales    |
+----+----------+
```
Write a SQL query to find employees who earn the top three salaries in each of the department. For the above tables, your SQL query should return the following rows (order of rows does not matter).
```
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Randy    | 85000  |
| IT         | Joe      | 85000  |
| IT         | Will     | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |
+------------+----------+--------+
```
Explanation:

In IT department, Max earns the highest salary, both Randy and Joe earn the second highest salary, and Will earns the third highest salary. There are only two employees in the Sales department, Henry earns the highest salary while Sam earns the second highest salary.



```mysql
select d.Name 'Department', e1.Name 'Employee', e1.Salary
from Employee e1
    join
    Department d on e1.DepartmentId = d.Id
where
    3 > (select count(distinct e2.salary)
    from Employee e2
    where e2.salary > e1.salary
        and e1.DepartmentId = e2.DepartmentId);
```

## Rank Scores (Medium)
Write a SQL query to rank scores. If there is a tie between two scores, both should have the same ranking. Note that after a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no "holes" between ranks.

+----+-------+
| Id | Score |
+----+-------+
| 1  | 3.50  |
| 2  | 3.65  |
| 3  | 4.00  |
| 4  | 3.85  |
| 5  | 4.00  |
| 6  | 3.65  |
+----+-------+
For example, given the above `Scores` table, your query should generate the following report (order by highest score):

+-------+---------+
| score | Rank    |
+-------+---------+
| 4.00  | 1       |
| 4.00  | 1       |
| 3.85  | 2       |
| 3.65  | 3       |
| 3.65  | 3       |
| 3.50  | 4       |
+-------+---------+
Important Note: For MySQL solutions, to escape reserved words used as column names, you can use an apostrophe before and after the keyword. For example `Rank`.

Solution1:
```mysql
select score,
    dense_rank() over (order by score desc) "Rank"
from scores
```

Soulution2:
```mysql
select score,
    (select count(*)
    from (select distinct score s
        from scores) tmp where s >= score) "Rank"
from scores
order by score desc;
```

## 181. Employees Earning More than Their Manager (Easy)
The `Employee` table holds all employees including their managers. Every employee has an Id, and there is also a column for the manager Id.
```
+----+-------+--------+-----------+
| Id | Name  | Salary | ManagerId |
+----+-------+--------+-----------+
| 1  | Joe   | 70000  | 3         |
| 2  | Henry | 80000  | 4         |
| 3  | Sam   | 60000  | NULL      |
| 4  | Max   | 90000  | NULL      |
+----+-------+--------+-----------+
```
Given the Employee table, write a SQL query that finds out employees who earn more than their managers. For the above table, Joe is the only employee who earns more than his manager.
```
+----------+
| Employee |
+----------+
| Joe      |
+----------+
```

```mysql
select a.name "Employee"
from Employee a
join Employee b on a.ManagerId = b.Id
where a.salary > b.salary;
```


## 262. Trips and Users (Hard)
The `Trips` table holds all taxi trips. Each trip has a unique Id, while Client_Id and Driver_Id are both foreign keys to the Users_Id at the Users table. Status is an ENUM type of (‘completed’, ‘cancelled_by_driver’, ‘cancelled_by_client’).
```
+----+-----------+-----------+---------+--------------------+----------+
| Id | Client_Id | Driver_Id | City_Id |        Status      |Request_at|
+----+-----------+-----------+---------+--------------------+----------+
| 1  |     1     |    10     |    1    |     completed      |2013-10-01|
| 2  |     2     |    11     |    1    | cancelled_by_driver|2013-10-01|
| 3  |     3     |    12     |    6    |     completed      |2013-10-01|
| 4  |     4     |    13     |    6    | cancelled_by_client|2013-10-01|
| 5  |     1     |    10     |    1    |     completed      |2013-10-02|
| 6  |     2     |    11     |    6    |     completed      |2013-10-02|
| 7  |     3     |    12     |    6    |     completed      |2013-10-02|
| 8  |     2     |    12     |    12   |     completed      |2013-10-03|
| 9  |     3     |    10     |    12   |     completed      |2013-10-03|
| 10 |     4     |    13     |    12   | cancelled_by_driver|2013-10-03|
+----+-----------+-----------+---------+--------------------+----------+
```
The `Users` table holds all users. Each user has an unique Users_Id, and Role is an ENUM type of (‘client’, ‘driver’, ‘partner’).
```
+----------+--------+--------+
| Users_Id | Banned |  Role  |
+----------+--------+--------+
|    1     |   No   | client |
|    2     |   Yes  | client |
|    3     |   No   | client |
|    4     |   No   | client |
|    10    |   No   | driver |
|    11    |   No   | driver |
|    12    |   No   | driver |
|    13    |   No   | driver |
+----------+--------+--------+
```
Write a SQL query to find the cancellation rate of requests made by unbanned users (both client and driver must be unbanned) between Oct 1, 2013 and Oct 3, 2013. The cancellation rate is computed by dividing the number of canceled (by client or driver) requests made by unbanned users by the total number of requests made by unbanned users.

For the above tables, your SQL query should return the following rows with the cancellation rate being rounded to two decimal places.
```
+------------+-------------------+
|     Day    | Cancellation Rate |
+------------+-------------------+
| 2013-10-01 |       0.33        |
| 2013-10-02 |       0.00        |
| 2013-10-03 |       0.50        |
+------------+-------------------+
```


```mysql
select request_at as Day,
    round(count(if(status != 'completed', true, null)) / count(*), 2) as 'Cancellation Rate'
from trips
where client_id in (select users_id from users where Banned='No')
    and driver_id in (select users_id from users where Banned='No')
    and request_at between '2013-10-01' and '2013-10-03'
group by 1;
```

## 196. Delete Duplicate Emails (Easy)
Write a SQL query to delete all duplicate email entries in a table named `Person`, keeping only unique emails based on its smallest Id.
```
+----+------------------+
| Id | Email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |
+----+------------------+
```
Id is the primary key column for this table.
For example, after running your query, the above `Person` table should have the following rows:
```
+----+------------------+
| Id | Email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
+----+------------------+
```
Note:

Your output is the whole Person table after executing your sql. Use `delete` statement

```mysql
delete p1
from person p1, person p2
where p1.Email = p2.Email and p1.id > p2.id;
```



## 184. Department Highest Salary (Medium)
The `Employee` table holds all employees. Every employee has an Id, a salary, and there is also a column for the department Id.
```
+----+-------+--------+--------------+
| Id | Name  | Salary | DepartmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 70000  | 1            |
| 2  | Jim   | 90000  | 1            |
| 3  | Henry | 80000  | 2            |
| 4  | Sam   | 60000  | 2            |
| 5  | Max   | 90000  | 1            |
+----+-------+--------+--------------+
```
The `Department` table holds all departments of the company.
```
+----+----------+
| Id | Name     |
+----+----------+
| 1  | IT       |
| 2  | Sales    |
+----+----------+
```
Write a SQL query to find employees who have the highest salary in each of the departments. For the above tables, your SQL query should return the following rows (order of rows does not matter).
```
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Jim      | 90000  |
| Sales      | Henry    | 80000  |
+------------+----------+--------+
```
Explanation:

Max and Jim both have the highest salary in the IT department and Henry has the highest salary in the Sales department.


```mysql
select d.name as 'Department', e.name as 'Employee', e.salary as 'Salary'
from employee e
join Department d on e.DepartmentId = d.id
where (e.DepartmentId, e.Salary) in (
    select DepartmentId, max(Salary)
    from employee
    group by DepartmentId
);
```


## 1179. Reformat Department Table (Easy)
Table: `Department`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| revenue       | int     |
| month         | varchar |
+---------------+---------+
```
(id, month) is the primary key of this table.
The table has information about the revenue of each department per month.
The month has values in ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"].


Write an SQL query to reformat the table such that there is a department id column and a revenue column for each month.

The query result format is in the following example:

`Department` table:
```
+------+---------+-------+
| id   | revenue | month |
+------+---------+-------+
| 1    | 8000    | Jan   |
| 2    | 9000    | Jan   |
| 3    | 10000   | Feb   |
| 1    | 7000    | Feb   |
| 1    | 6000    | Mar   |
+------+---------+-------+
```
Result table:
```
+------+-------------+-------------+-------------+-----+-------------+
| id   | Jan_Revenue | Feb_Revenue | Mar_Revenue | ... | Dec_Revenue |
+------+-------------+-------------+-------------+-----+-------------+
| 1    | 8000        | 7000        | 6000        | ... | null        |
| 2    | 9000        | null        | null        | ... | null        |
| 3    | null        | 10000       | null        | ... | null        |
+------+-------------+-------------+-------------+-----+-------------+
```

Note that the result table has 13 columns (1 for the department id + 12 for the months).



```mysql
select
    id,
    max(if(month='jan', revenue, null)) as Jan_Revenue,
    max(if(month='Feb', revenue, null)) as Feb_Revenue,
    max(if(month='Mar', revenue, null)) as Mar_Revenue,
    max(if(month='Apr', revenue, null)) as Apr_Revenue,
    max(if(month='May', revenue, null)) as May_Revenue,
    max(if(month='Jun', revenue, null)) as Jun_Revenue,
    max(if(month='Jul', revenue, null)) as Jul_Revenue,
    max(if(month='Aug', revenue, null)) as Aug_Revenue,
    max(if(month='Sep', revenue, null)) as Sep_Revenue,
    max(if(month='Oct', revenue, null)) as Oct_Revenue,
    max(if(month='Nov', revenue, null)) as Nov_Revenue,
    max(if(month='Dec', revenue, null)) as Dec_Revenue
from department
group by id;
```


## 626. Exchange Seats (Medium)
Mary is a teacher in a middle school and she has a table seat storing students' names and their corresponding seat ids.

The column id is continuous increment.


Mary wants to change seats for the adjacent students.


Can you write a SQL query to output the result for Mary?

```
+---------+---------+
|    id   | student |
+---------+---------+
|    1    | Abbot   |
|    2    | Doris   |
|    3    | Emerson |
|    4    | Green   |
|    5    | Jeames  |
+---------+---------+
```
For the sample input, the output is:

```
+---------+---------+
|    id   | student |
+---------+---------+
|    1    | Doris   |
|    2    | Abbot   |
|    3    | Green   |
|    4    | Emerson |
|    5    | Jeames  |
+---------+---------+
```
Note:
If the number of students is odd, there is no need to change the last one's seat.



Solution1 (case):
```mysql
select
    (case
        when mod(id, 2) != 0 and counts != id then id+1
        when mod(id, 2) != 0 and counts = id then id
        else id -1
    end) id,
    student
from
    seat,
    (select count(*) as counts
    from seat) as seat_counts
order by id asc;
```

Solution2 (coalesce, bit manipulation):
```mysql
select s1.id, coalesce(s2.student, s1.student) as tudent
from seat s1
left join seat s2 on ((s1.id+1)^1)-1 = s2.id
order by s1.id asc;
````

## 180. Consecutive Numbers (Medium)
Write a SQL query to find all numbers that appear at least three times consecutively.
```
+----+-----+
| Id | Num |
+----+-----+
| 1  |  1  |
| 2  |  1  |
| 3  |  1  |
| 4  |  2  |
| 5  |  1  |
| 6  |  2  |
| 7  |  2  |
+----+-----+
```
For example, given the above `Logs` table, 1 is the only number that appears consecutively for at least three times.
```
+-----------------+
| ConsecutiveNums |
+-----------------+
| 1               |
+-----------------+
```



```mysql
select l1.num 'ConsecutiveNumbs'
from logs l1, logs l2, logs l3
where l1.id = l2.id-1
    and l2.id = l3.id-1
    and l1.num = l2.num
    and l2.num = l3.num;
```



## 615. Average Salary: Departments VS Company (Hard)
Given two tables as below, write a query to display the comparison result (higher/lower/same) of the average salary of employees in a department to the company's average salary.


Table: `salary`
```
| id | employee_id | amount | pay_date   |
|----|-------------|--------|------------|
| 1  | 1           | 9000   | 2017-03-31 |
| 2  | 2           | 6000   | 2017-03-31 |
| 3  | 3           | 10000  | 2017-03-31 |
| 4  | 1           | 7000   | 2017-02-28 |
| 5  | 2           | 6000   | 2017-02-28 |
| 6  | 3           | 8000   | 2017-02-28 |
```

The employee_id column refers to the employee_id in the following table `employee`.

```
| employee_id | department_id |
|-------------|---------------|
| 1           | 1             |
| 2           | 2             |
| 3           | 2             |
```

So for the sample data above, the result is:

```
| pay_month | department_id | comparison  |
|-----------|---------------|-------------|
| 2017-03   | 1             | higher      |
| 2017-03   | 2             | lower       |
| 2017-02   | 1             | same        |
| 2017-02   | 2             | same        |
```

```mysql
select d.pay_month, d.department_id,
    (case
        when d.department_avg > c.company_avg then 'higher'
        when d.department_avg < c.company_avg then 'lower'
        else 'same'
    end) comparison
from
    (select e.department_id, avg(s.amount) department_avg, date_format(s.pay_date, '%Y-%m') pay_month
    from salary s
    join employee e on s.employee_id = e.employee_id
    group by e.department_id, pay_month) d
join
    (select avg(amount) company_avg, date_format(pay_date, '%Y-%m') pay_month
    from salary
    group by pay_month) c
on d.pay_month = c.pay_month;
```



## 601. Human Traffic of Stadium (Hard)
Table: `Stadium`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| visit_date    | date    |
| people        | int     |
+---------------+---------+
```
visit_date is the primary key for this table.
Each row of this table contains the visit date and visit id to the stadium with the number of people during the visit.
No two rows will have the same visit_date, and as the id increases, the dates increase as well.


Write an SQL query to display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by visit_date in ascending order.

The query result format is in the following example.



`Stadium` table:
```
+------+------------+-----------+
| id   | visit_date | people    |
+------+------------+-----------+
| 1    | 2017-01-01 | 10        |
| 2    | 2017-01-02 | 109       |
| 3    | 2017-01-03 | 150       |
| 4    | 2017-01-04 | 99        |
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |
+------+------------+-----------+
```

Result table:
```
+------+------------+-----------+
| id   | visit_date | people    |
+------+------------+-----------+
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |
+------+------------+-----------+
```
The four rows with ids 5, 6, 7, and 8 have consecutive ids and each of them has >= 100 people attended. Note that row 8 was included even though the visit_date was not the next day after row 7.
The rows with ids 2 and 3 are not included because we need at least three consecutive ids.




```mysql
select distinct t1.*
from stadium t1, stadium t2, stadium t3
where t1.people >= 100 and t2.people >= 100 and t3.people>=100
    and (  -- only the order of t1 is considered
            (t1.id = t2.id - 1 and t1.id = t3.id - 2) -- t1 < t2 < t3
        or
            (t3.id = t1.id - 1 and t3.id = t2.id - 2) -- t3 < t1 < t2
        or
            (t3.id = t2.id - 1 and t3.id = t1.id - 2) -- t3 < t2 < t1
        )
order by t1.id;
```



## 1270. All People Report to the Given Manager (Medium)
Table: `Employees`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| employee_id   | int     |
| employee_name | varchar |
| manager_id    | int     |
+---------------+---------+
```
employee_id is the primary key for this table.
Each row of this table indicates that the employee with ID employee_id and name employee_name reports his work to his/her direct manager with manager_id
The head of the company is the employee with employee_id = 1.


Write an SQL query to find employee_id of all employees that directly or indirectly report their work to the head of the company.

The indirect relation between managers will not exceed 3 managers as the company is small.

Return result table in any order without duplicates.

The query result format is in the following example:



```mysql
select e1.employee_id
from employees e1, employees e2, employees e3
where e1.manager_id = e2.employee_id
    and e2.manager_id = e3.employee_id
    and e3.manager_id = 1
    and e1.employee_id != 1;
```



## 1384. Total Sales Amount by Year (Hard)
Table: `Product`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| product_name  | varchar |
+---------------+---------+
```
product_id is the primary key for this table.
product_name is the name of the product.


Table: `Sales`
```
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| product_id          | int     |
| period_start        | varchar |
| period_end          | date    |
| average_daily_sales | int     |
+---------------------+---------+
```
product_id is the primary key for this table.
period_start and period_end indicates the start and end date for sales period, both dates are inclusive.
The average_daily_sales column holds the average daily sales amount of the items for the period.

Write an SQL query to report the Total sales amount of each item for each year, with corresponding product name, product_id, product_name and report_year.

Dates of the sales years are between 2018 to 2020. Return the result table ordered by product_id and report_year.

The query result format is in the following example:


`Product` table:
```
+------------+--------------+
| product_id | product_name |
+------------+--------------+
| 1          | LC Phone     |
| 2          | LC T-Shirt   |
| 3          | LC Keychain  |
+------------+--------------+
```

`Sales` table:
```
+------------+--------------+-------------+---------------------+
| product_id | period_start | period_end  | average_daily_sales |
+------------+--------------+-------------+---------------------+
| 1          | 2019-01-25   | 2019-02-28  | 100                 |
| 2          | 2018-12-01   | 2020-01-01  | 10                  |
| 3          | 2019-12-01   | 2020-01-31  | 1                   |
+------------+--------------+-------------+---------------------+
```
Result table:
```
+------------+--------------+-------------+--------------+
| product_id | product_name | report_year | total_amount |
+------------+--------------+-------------+--------------+
| 1          | LC Phone     |    2019     | 3500         |
| 2          | LC T-Shirt   |    2018     | 310          |
| 2          | LC T-Shirt   |    2019     | 3650         |
| 2          | LC T-Shirt   |    2020     | 10           |
| 3          | LC Keychain  |    2019     | 31           |
| 3          | LC Keychain  |    2020     | 31           |
+------------+--------------+-------------+--------------+
```
LC Phone was sold for the period of 2019-01-25 to 2019-02-28, and there are 35 days for this period. Total amount 35*100 = 3500.
LC T-shirt was sold for the period of 2018-12-01 to 2020-01-01, and there are 31, 365, 1 days for years 2018, 2019 and 2020 respectively.
LC Keychain was sold for the period of 2019-12-01 to 2020-01-31, and there are 31, 31 days for years 2019 and 2020 respectively.



```mysql
select a.product_id, b.product_name, a.report_year, a.total_amount
from (
    select product_id, '2018' as report_year,
        average_daily_sales * (datediff(least(period_end, '2018-12-31'), greatest(period_start, '2018-01-01'))+1) as total_amount
    from sales
    where year(period_start)=2018 or year(period_end)=2018

    union all

    select product_id, '2019' as report_year,
        average_daily_sales * (datediff(least(period_end, '2019-12-31'), greatest(period_start, '2019-01-01'))+1) as total_amount
    from sales
    where year(period_start)<=2019 and year(period_end)>=2019 -- becareful!! <= & >=

    union all

    select product_id, '2020' as report_year,
        average_daily_sales * (datediff(least(period_end, '2020-12-31'), greatest(period_start, '2020-01-01'))+1) as total_amount
    from sales
    where year(period_start)=2020 or year(period_end)=2020
) a
left join product b
on a.product_id = b.product_id
order by a.product_id, a.report_year;
```



## 182. Duplicate Emails (Easy)
Write a SQL query to find all duplicate emails in a table named `Person`.
```
+----+---------+
| Id | Email   |
+----+---------+
| 1  | a@b.com |
| 2  | c@d.com |
| 3  | a@b.com |
+----+---------+
```
For example, your query should return the following for the above table:
```
+---------+
| Email   |
+---------+
| a@b.com |
+---------+
```
Note: All emails are in lowercase.

```mysql
select email
from person
group by email
having count(email) > 1;
```